<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Création du dataset "Trésorerie"

**Tags:** #fec #finance #snippet #operation #trésorerie #tft

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** Ce notebook permet de créer le dataframe pour visualiser l'évolution de la trésorerie avec les entrées et sorties de cash et la position à travers les mois.

## Input

### Import libraries

In [1]:
import naas_data_product

✅ utils file '/home/ftp/FEC-engine/utils/datalake.ipynb' successfully loaded.


### Setup Variables

In [2]:
# Inputs
input_folder_path = "/home/ftp/FEC-engine/outputs/FEC/bdd_fin"

# Outputs
output_folder_path = "/home/ftp/FEC-engine/outputs/FEC/dataset_tresorerie"

## Model

### Récupération du dernier fichier input

In [3]:
df_input = get_last_df(input_folder_path)
print("✅ Row fetched:", len(df_input))
df_input.head(1)

📁 Last file: /home/ftp/FEC-engine/outputs/FEC/bdd_fin/20230601154259_FEC_bdd_fin.csv
✅ Row fetched: 4532


,NOM_FICHIER,ENTITY,PERIOD,DATE,MONTH,MONTH_INDEX,COMPTE_NUM,RUBRIQUE_N3,RUBRIQUE_N4,DEBIT,CREDIT,VALUE
0,000000000FEC20181231.txt,000000000,201812,2018-01-01,Jan,1,20500000,"BREVETS, LICENCES, LOGICIELS..",A Nouveau,4006.6,0.0,4006.6


### Création du dataset "TRESORERIE"

In [4]:
# Creation du dataset trésorerie
dataset_treso = df_input.copy()

# Filtre RUBRIQUE_N1 = TRESORERIE
dataset_treso = dataset_treso[
    dataset_treso["COMPTE_NUM"].str.contains(r"^5")
].reset_index(drop=True)

# Cash in / Cash out ?
dataset_treso.loc[dataset_treso.VALUE > 0, "CASH_IN"] = dataset_treso.VALUE
dataset_treso.loc[dataset_treso.VALUE < 0, "CASH_OUT"] = dataset_treso.VALUE

# Regroupement
to_group = ["ENTITY", "PERIOD", "MONTH", "MONTH_INDEX"]
to_agg = {"VALUE": "sum", "CASH_IN": "sum", "CASH_OUT": "sum"}
dataset_treso = dataset_treso.groupby(to_group, as_index=False).agg(to_agg).fillna(0)

# Cumul par période
dataset_treso = dataset_treso.sort_values(["ENTITY", "PERIOD", "MONTH_INDEX"])
dataset_treso["MONTH_INDEX"] = pd.to_datetime(
    dataset_treso["MONTH_INDEX"], format="%m"
).dt.strftime("%m")
dataset_treso["VALUE_LINE"] = dataset_treso.groupby(
    ["ENTITY", "PERIOD"], as_index=True
).agg({"VALUE": "cumsum"})

# Mettre en valeur positive CASH_OUT
dataset_treso["CASH_OUT"] = dataset_treso["CASH_OUT"].abs()

# Formatting
dataset_treso["VALUE_D"] = (dataset_treso["VALUE"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
dataset_treso["CASH_IN_D"] = (dataset_treso["CASH_IN"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
dataset_treso["CASH_OUT_D"] = (dataset_treso["CASH_OUT"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
dataset_treso["VALUE_LINE_D"] = (dataset_treso["VALUE_LINE"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")

# Affichage du modèle de donnée
dataset_treso

,ENTITY,PERIOD,MONTH,MONTH_INDEX,VALUE,CASH_IN,CASH_OUT,VALUE_LINE,VALUE_D,CASH_IN_D,CASH_OUT_D,VALUE_LINE_D
4,000000000,201712,Jan,01,129503.90,168471.60,38967.70,129503.90,129.5 k€,168.5 k€,39.0 k€,129.5 k€
3,000000000,201712,Feb,02,-6594.49,42205.15,48799.64,122909.41,-6.6 k€,42.2 k€,48.8 k€,122.9 k€
7,000000000,201712,Mar,03,267.46,41521.46,41254.00,123176.87,0.3 k€,41.5 k€,41.3 k€,123.2 k€
0,000000000,201712,Apr,04,-60412.14,65481.71,125893.85,62764.73,-60.4 k€,65.5 k€,125.9 k€,62.8 k€
8,000000000,201712,May,05,7139.11,119644.49,112505.38,69903.84,7.1 k€,119.6 k€,112.5 k€,69.9 k€
6,000000000,201712,Jun,06,26363.71,44719.45,18355.74,96267.55,26.4 k€,44.7 k€,18.4 k€,96.3 k€
5,000000000,201712,Jul,07,16816.88,172363.85,155546.97,113084.43,16.8 k€,172.4 k€,155.5 k€,113.1 k€
1,000000000,201712,Aug,08,170.32,23093.20,22922.88,113254.75,0.2 k€,23.1 k€,22.9 k€,113.3 k€
11,000000000,201712,Sep,09,-3680.61,14201.45,17882.06,109574.14,-3.7 k€,14.2 k€,17.9 k€,109.6 k€
10,000000000,201712,Oct,10,-3668.05,43058.55,46726.60,105906.09,-3.7 k€,43.1 k€,46.7 k€,105.9 k€


## Output

### Sauvegarde des fichiers en csv

In [5]:
save_df(dataset_treso, output_folder_path)

✅ DataFrame saved in: /home/ftp/FEC-engine/outputs/FEC/dataset_tresorerie/20230601155129_FEC_dataset_tresorerie.csv
